In [580]:
import math

route_1 = {
    
    'duration' : 40,
    
    'timeslots' : {
    
        1 : {
            'time_start' : 0,
            'time_end': 239,
            'frequency' : 10,
            'people' : 2000
            
        },
        
        2 : {
            'time_start' : 240,
            'time_end': 599,
            'frequency' : 10,
            'people' : 800
        },
        
        3 : {
            'time_start' : 600,
            'time_end': 839,
            'frequency' : 10,
            'people' : 2000
        },
        
        4 : {
            'time_start' : 840,
            'time_end': 1199,
            'frequency' : 5,
            'people' : 4000
        }
    
    }
    
}

In [654]:
def findCurrentTimeSlot(time):
    if time > 240: return 1
    elif time > 600: return 2
    elif time > 840: return 3
    elif time > 1200: return 4
    else: return 0

class Route: 
    
    def __init__(self, id, duration, time = 0, timeslots = None, frequency = 10, people = 800):
        if time == 0: self.current_timeslot = 1
        else: self.current_timeslot = findCurrentTimeSlot(time)
        self.timeslots = timeslots
        self.id = id
        self.duration = duration
        self.last_start = -1000
        self.frequency = frequency
        self.people = people
        
    
    def setTimesplots(self, timeslots):
        self.timeslots = timeslots
        
    def update(self, time):
        
        # update the time
        if (time >= self.timeslots[self.current_timeslot]['time_end']):
            self.current_timeslot += 1
        if self.current_timeslot == 5 : self.current_timeslot = 1 
          
        # update data
        self.frequency = self.timeslots[self.current_timeslot]['frequency']
        self.people = self.timeslots[self.current_timeslot]['people']
            
    def shouldStart(self, time):
        return time - self.last_start >= self.frequency
        

In [655]:
class Bus: 
    
    def __init__ (self, id, capacity = 20, battery = None, route = None):
        self.id = id
        self.battery = battery
        self.current_route = route
        self.remaining_time = 0
        self.isRiding = False
        self.capacity = capacity
    
        
        
    def update(self):
        
        if not self.isRiding :
            print ("Bus " + str(self.id)+ " waiting at the station")
            return
        
        self.remaining_time -= 1
        self.battery.use(self.id)
        
        print ("Bus " + str(self.id)+ " have still " + str(self.remaining_time) + " minutes to drive")
        if self.remaining_time == 0:
            print ("Bus " + str(self.id) + " is done, it is now at the station")
            self.done()
        
    def start(self, route, time):
        self.current_route = route
        self.remaining_time = route.duration
        self.isRiding = True
        station.leave(self)
        route.last_start = time
        
    def done(self):
        self.current_route = None
        self.isRiding = False
        station.add(self)
        
    def canTakeRoute(self, route):
        return (route.duration <= self.battery.power)


In [656]:
class Station:
    
    def __init__(self, buses = [], routes = []):
        self.buses = buses
        self.routes = routes
    
    def add(self, bus):
        self.buses.append(bus)
        
    def leave(self, bus):
        # it's done with pop in update
        # self.buses.remove(bus)
        pass
        
    def update(self, time):
        
        # routes dispatching
        for r in self.routes:
            
            if r.shouldStart(time):
                if (self.buses):
                    for b in self.buses:
                        if (b.canTakeRoute(r)):
                            self.buses.remove(b)
                            b.start(r, time)
                            print("Bus " + str(b.id) + " starts route " + str(r.id) )
                            break
                else:
                    print("Not enough buses at the station or with enough battery...")
        
        # Battery management
        
        for b in self.buses:                   
            # Charged
            if (b.battery.power >= b.battery.max_power):
                b.battery.power = b.battery.max_power
            else:
                # Charging
                b.battery.power += 0.017 * b.battery.max_power
                print("Bus " + str(b.id) + " fastly charging => " + str(math.floor(b.battery.power)) + "/" + str(b.battery.max_power))

        

In [657]:
class Battery:
    
    def __init__ (self, max_power, cost = 0):
        self.max_power = max_power
        self.power = max_power
        self.cost = cost
        self.nb_recharges = 0
        
    def charge (self, power): 
        self.power += power
        if (self.max_power < self.power):
            self.power = self.max_power
            
    def use(self, id):
        self.power -= 1
        print("Battery from bus " + str(id) + " has " + str(math.floor(self.power)) + " units of power left")
        if (self.power <= 0):
            print("FAIL!!! Bus ran out of energy while using")
            self.power = 0
        

In [658]:
class Charger : pass

# Fast chargin : 1h and 1h30
# one a minute basis, 1 tick is 1.7% && 1.1%

# Slow chargin : 

class FastCharger(Charger):
    
    def __init__ (self):
        self.current_bus = None
    
    def charge(self, bus):
        self.current_bus = bus
        
    def stopCharging(self):
        self.current_bus = None
        
    def update(self):
        if (self.current_bus):
            self.current_bus.battery.charge(1.7 * self.current_bus.battery.max_power)

In [659]:
class World: 
    
    def __init__ (self) : 
        self.time = 0
        self.chargers = [];
        self.buses = [];
        self.routes = [];
    
    def tick(self):
        
        self.time += 1
        
        print ("Time is now " + str(self.time))
        
        if (self.time >= 1200):
            self.time = 0
            
        for charger in self.chargers:
            charger.update()
        
        for bus in self.buses:
            bus.update()
            
        for route in self.routes:
            route.update(self.time)
            
        station.update(self.time)
   
        

In [660]:
r1 = Route(1, duration = 40, timeslots = route_1['timeslots'])
r2 = Route(2, duration = 60, timeslots = route_1['timeslots'])
r3 = Route(3, duration = 110, timeslots = route_1['timeslots'])

bt1 = Battery(130)
bt2 = Battery(80)
bt3 = Battery(100)
bt4 = Battery(100)
bt5 = Battery(100)
bt6 = Battery(100)
bt7 = Battery(100)
bt8 = Battery(100)

b1 = Bus(1, 20, bt1, r1)
b2 = Bus(2, 30, bt2, r1)
b3 = Bus(3, 50, bt3, r1)
b4 = Bus(4, 20, bt4, r1)
b5 = Bus(5, 20, bt5, r1)
b6 = Bus(6, 30, bt6, r1)
b7 = Bus(7, 50, bt7, r1)
b8 = Bus(8, 20, bt8, r1)

buses = [b1, b2, b3, b4, b5, b6, b7, b8]
routes = sorted([r1, r2, r3], key=lambda route: route.duration)[::-1]

station = Station(buses, routes)


In [661]:
w = World()

In [662]:
w.buses.extend(buses)
w.routes.extend(routes)


In [663]:
for i in range(300) : w.tick()

Time is now 1
Bus 1 waiting at the station
Bus 2 waiting at the station
Bus 3 waiting at the station
Bus 4 waiting at the station
Bus 5 waiting at the station
Bus 6 waiting at the station
Bus 7 waiting at the station
Bus 8 waiting at the station
Bus 1 starts route 3
Bus 2 starts route 2
Bus 3 starts route 1
Time is now 2
Battery from bus 1 has 129 units of power left
Bus 1 have still 109 minutes to drive
Battery from bus 2 has 99 units of power left
Bus 2 have still 59 minutes to drive
Battery from bus 3 has 99 units of power left
Bus 3 have still 39 minutes to drive
Bus 4 waiting at the station
Bus 5 waiting at the station
Bus 6 waiting at the station
Bus 7 waiting at the station
Bus 8 waiting at the station
Time is now 3
Battery from bus 1 has 128 units of power left
Bus 1 have still 108 minutes to drive
Battery from bus 2 has 98 units of power left
Bus 2 have still 58 minutes to drive
Battery from bus 3 has 98 units of power left
Bus 3 have still 38 minutes to drive
Bus 4 waiting at